https://www.kaggle.com/aniruddhachoudhury/sentiment-classification-with-nlp-on-lstm






https://www.kaggle.com/kshitijmohan/sentiment-analysis-universal-sentence-encoder-91






https://www.kaggle.com/saurav9786/recommendation-based-on-food-review






# Recommondation and Sentiment analysis on Amazon Fine food review dataset:

In [ ]:
from google.colab import drive   #connect collab to drive
drive.mount('/content/drive')    # mount drive

Mounted at /content/drive


In [ ]:
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pandas as pd
import matplotlib.pyplot as plt
import warnings 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Reviews.csv') #df is dataframe where train dataset is present.

# Recommendation of food products

In [ ]:
dataset=df.copy()               # uploading the dataset for recommendation purpose
dataset.head(5)                      

,Unnamed: 0,id,asins,brand,categories,dateAdded,dateUpdated,descriptions,dimension,ean,flavors,keys,manufacturer,manufacturerNumber,name,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sizes,sourceURLs,upc,weight
0,0,AV13ClKCGV-KLJ3akN68,NaN,Gallo,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:59:11Z,2018-01-10T18:06:28Z,NaN,1.0 in x 1.0 in x 1.0 in,NaN,NaN,"492130001994,gallo/13312834",NaN,13312834,Ecco Domani174 Pinot Grigio - 750ml Bottle,2017-08-01T21:13:49.000Z,2018-01-09T13:24:04Z,"2017-12-14T19:41:00.000Z,2017-12-19T19:55:00.0...",NaN,True,NaN,1.0,5.0,https://redsky.target.com/groot-domain-api/v1/...,This a fantastic white wine for any occasion!,My Favorite White Wine,NaN,NaN,Bjh,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,4.92E+11,1.0 lbs
1,1,AV13CsvW-jtxr-f38AQO,NaN,Fresh Craft Co.,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:59:42Z,2018-01-10T05:38:33Z,"[{""dateSeen"":[""2017-12-21T05:43:00.000Z"",""2017...",4.25 in x 4.25 in x 5.25 in,NaN,NaN,"freshcraft/50392800,083120003441",NaN,50392800,Fresh Craft174 Mango Citrus - 4pk / 250ml Bottle,2017-07-10T04:17:34.000Z,2018-01-09T17:31:52Z,"2017-12-17T05:32:00.000Z,2017-12-21T14:32:00.0...",NaN,True,NaN,NaN,5.0,https://redsky.target.com/groot-domain-api/v1/...,"Tart, not sweet...very refreshing and delicious!",Yum!!,NaN,NaN,Wino,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,83120003441,2.45 lbs
2,2,AV13CVI_glJLPUi8O7Po,NaN,1000 Stories,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:58:05Z,2018-01-10T05:38:31Z,NaN,3.3 in x 3.3 in x 11.79 in,NaN,NaN,"082896001453,1000stories/50399893",NaN,50399893,1000 Stories174 Zinfandel - 750ml Bottle,2017-10-02T12:52:27.000Z,2018-01-09T17:31:51Z,"2017-12-17T05:30:00.000Z,2017-12-21T14:29:00.0...",NaN,True,NaN,NaN,5.0,https://redsky.target.com/groot-domain-api/v1/...,I was given this wine so it was a delightful s...,A New Favorite!,NaN,NaN,Bama Mom,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,82896001453,3.09 lbs
3,3,AV13CVI_glJLPUi8O7Po,NaN,1000 Stories,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:58:05Z,2018-01-10T05:38:31Z,NaN,3.3 in x 3.3 in x 11.79 in,NaN,NaN,"082896001453,1000stories/50399893",NaN,50399893,1000 Stories174 Zinfandel - 750ml Bottle,2016-11-02T10:00:37.000Z,2017-10-04T18:03:12Z,"2017-09-30T00:52:32.856Z,2017-08-08T10:33:31.6...",NaN,True,NaN,NaN,5.0,https://redsky.target.com/groot-domain-api/v1/...,This is a phenomenal wine and my new favorite ...,"Bold, Flavorful, Aromatic, Delicious",NaN,NaN,Av Dub,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,82896001453,3.09 lbs
4,4,AV13CYL4-jtxr-f37_-t,NaN,Wine Cube153,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:58:18Z,2018-01-10T18:06:29Z,"[{""dateSeen"":[""2017-12-21T05:43:00.000Z"",""2017...",1.0 in x 1.0 in x 1.0 in,NaN,NaN,"winecube153/14831433,085200600465",NaN,14831433,Pink Moscato - 3l Bottle - Wine Cube153,2014-11-25T03:37:34.000Z,2018-01-10T18:06:29Z,"2017-12-16T09:19:00.000Z,2017-12-20T16:57:00.0...",NaN,True,NaN,1.0,5.0,https://redsky.target.com/groot-domain-api/v1/...,4 750ml bottles for the price of two With way ...,"Yum! Plus, Environmentally Friendly!",Overland Park,NaN,Chelseamay,NaN,https://redsky.target.com/groot-domain-api/v1/...,85200600465,1.0 lbs


In [ ]:
newDF=dataset.drop(columns=['Id','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Time','Summary','Text'])  # dropping the columns that are of no use
newDF=newDF.drop_duplicates()    # dropped the duplicates
newDF.head(5)

In [ ]:
pd_series=newDF.Score.value_counts()         # count of the number of rating of each type 
ratings=[5,4,1,3,2]
count_ratings=[]
for i in pd_series:
    count_ratings.append(i)
plt.bar(ratings,count_ratings)
print(pd_series)
plt.xlabel("ratings")
plt.ylabel("number of records")
plt.show()

In [ ]:
print(newDF['UserId'].nunique())             # counting the number of unique users
print(newDF['ProductId'].nunique())          # counting the number of unique products

In [ ]:
# we can see that there are many user but very less product, so we will keep the product which is rated rated by atleast 30 users
counts=newDF.ProductId.value_counts()
df_final=newDF[newDF['ProductId'].isin(counts[counts>=30].index)]
counts=df_final.UserId.value_counts()    # also we kept only that user which has rated atleast 10 movies
df_final=df_final[df_final['UserId'].isin(counts[counts>=10].index)]
df_final.shape

### Building Popularity Recommender for new users

In [ ]:
popular_products=pd.DataFrame(newDF.groupby('ProductId')['Score'].count())   # here we selected the product having highest number of ratings 
most_popular_products=popular_products.sort_values('Score',ascending=False)
most_popular_products_10=most_popular_products[:10]
print('10 most popular products are:')
print(most_popular_products_10)
most_popular_products_10.plot(kind = "bar")
plt.ylabel('number of reviews')

In [ ]:
x=most_popular_products_10.index                    # printing the most popular product which is to be shown for the new users
most_popular_food_brand=[]
for i in x:
    most_popular_food_brand.append(i)
most_popular_food_brand

### Collaborative filtering on food items

In [ ]:
corrMatrix=matrix.corr(method='pearson')    # finding the correlation between each product items
corrMatrix

In [ ]:
def Recommendation(userList,most_popular_food_item,matrix,corrMatrix):
    for i in userList:                                                   
        print(' ')
        print("recommendation for user {} is:".format(i))
        if i in matrix.index:
            myRatings=matrix.loc[i].dropna()                                         # selecting all the product which has been rated by user i 
            simCandidates=pd.Series()  
            for j in range(0,len(myRatings.index)):                                  
                #print(myRatings.index[j])
                sims=corrMatrix[myRatings.index[j]].dropna()                         # selecting the row in correlation matrix which user has rated
                sims=sims.map(lambda x: x*myRatings[j])                              # multiplying the correlation coefficient with the ratings so that highly rated product values goes high
                simCandidates=simCandidates.append(sims)                             # appending all the rated product by the user i
            simCandidates.sort_values(inplace=True,ascending=False)                  # now sorting all the value in descending order
            simCandidates=simCandidates.groupby(simCandidates.index).sum()           # we summed the rating of each product because there might be scenario where more than one product by user has some common product
            simCandidates.sort_values(inplace=True,ascending=False)                  # now we sorted it again for product having high summed ratings
            recommended_product=[] 
            count=0
            for j in simCandidates.index:                                            # in this we have printed the top10 product excluding the product rated by user itself
                if j not in myRatings.index and count<10:
                    count+=1
                    print(j)
        else:                                                                        # recommendation for new users
            for i in most_popular_food_item:
                print(i)

In [ ]:
warnings.filterwarnings("ignore")
usersList=['AY12DBB0U420B','A2SZLNSI5KOQJT','A3OXHLG6DIBRW8','0','1']           # user ids having existing user and new users
Recommendation(usersList,most_popular_food_brand,matrix,corrMatrix)

# Sentiment Analysis of the product reviews:

## Tfidf-vectorization and Count-vectorization in reviews:

In [ ]:
df.columns  #name of all thge columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

Data prepration:

In [ ]:
df['Text']=df['Text'].fillna("")      #fill null values with "": empty string
df['Score']=df['Score'].fillna(0.0)    #fill null values with 0.0

In [ ]:
df.head(4)   #whole dataset

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...


import all the nltk libereries:

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['Text']   #text columns

0         I have bought several of the Vitality canned d...
1         Product arrived labeled as Jumbo Salted Peanut...
2         This is a confection that has been around a fe...
3         If you are looking for the secret ingredient i...
4         Great taffy at a great price.  There was a wid...
                                ...                        
568449    Great for sesame chicken..this is a good if no...
568450    I'm disappointed with the flavor. The chocolat...
568451    These stars are small, so you can give 10-15 o...
568452    These are the BEST treats for training and rew...
568453    I am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object

In [ ]:
df['Score'].value_counts()    # rating columns name Score:

5    363122
4     80655
1     52268
3     42640
2     29769
Name: Score, dtype: int64

In [ ]:
 re.sub('[^a-zA-Z]', ' ', df['Text'][0])   #substitiude non alphabets with spaces 

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality  The product looks more like a stew than a processed meat and it smells better  My Labrador is finicky and she appreciates this product better than  most '

In [ ]:
df['Text']

0         I have bought several of the Vitality canned d...
1         Product arrived labeled as Jumbo Salted Peanut...
2         This is a confection that has been around a fe...
3         If you are looking for the secret ingredient i...
4         Great taffy at a great price.  There was a wid...
                                ...                        
568449    Great for sesame chicken..this is a good if no...
568450    I'm disappointed with the flavor. The chocolat...
568451    These stars are small, so you can give 10-15 o...
568452    These are the BEST treats for training and rew...
568453    I am very satisfied ,product is as advertised,...
Name: Text, Length: 568454, dtype: object

Data preprocessing:

In [ ]:
# import random
# Here data prperocessing is happened:
corpus = []
for i in range(0,50000):
    # print( df['Text'][x])
    review = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '',df['Text'][i], flags=re.MULTILINE)   #remove url
    review = re.sub('[^a-zA-Z]', ' ', review)   #remove numberic or other then aphabets to spaces

    review = review.lower()    #upper to lower case conversion
    review = review.split()      #split sentences 
    ps = PorterStemmer()          #applied porter stemmer
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]   #removing stopwords
    review = ' '.join(review)
    corpus.append(review)      #all the preprocessed reviews are stored in corpus list and further put in Reviews column 

In [ ]:
corpus=pd.DataFrame(corpus, columns=['Reviews'])  #put copus values into corpus dataframe
corpus.head()

,Reviews
0,bought sever vital can dog food product found ...
1,product arriv label jumbo salt peanut peanut a...
2,confect around centuri light pillowi citru gel...
3,look secret ingredi robitussin believ found go...
4,great taffi great price wide assort yummi taff...


In [ ]:
result=corpus.join(df[['Score']])    #join the score column to the result and now result is now my new dataset
result.head()

,Reviews,Score
0,bought sever vital can dog food product found ...,5
1,product arriv label jumbo salt peanut peanut a...,1
2,confect around centuri light pillowi citru gel...,4
3,look secret ingredi robitussin believ found go...,2
4,great taffi great price wide assort yummi taff...,5


**TF-idf vectorization**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()      #tfidf vectorization 
tfidf.fit(result['Reviews'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
doc_vector_tfidf= tfidf.transform(result['Reviews'])   #transform the tfidf vectorization of Reviews column of result.
result['Reviews'][1]

'product arriv label jumbo salt peanut peanut actual small size unsalt sure error vendor intend repres product jumbo'

In [ ]:
doc_vector_tfidf   # this is final matrix of tf-idf vectorization with 50000 x 25419 columns

<50000x25419 sparse matrix of type '<class 'numpy.float64'>'
	with 1597243 stored elements in Compressed Sparse Row format>

In [ ]:
print([doc_vector_tfidf[1, tfidf.vocabulary_['peanut']]]) #peanuts tfidf value is 0.33

[0.33186936733548433]


In [ ]:
print([doc_vector_tfidf[1, tfidf.vocabulary_['jumbo']]])   # jumbo tfidf is 0.604 so here 
                                                           # jumbo is rare word having high tf-idfvalue:

[0.6042910499832276]


In [ ]:
import numpy as np


label encode by converting all the labesls greater then 3 to class postive that is 1 else labels all of them to 0 as negative:

In [ ]:
cor=[]
result.dropna(inplace=True)     
for i in result['Score']:      #label encode by converting all the labesls greater then 3 to class
  # if i == 3.0:               # postive that is 1 else labels all of them to 0 as negative:
  #   cor.append(2)
  if i>3.0:
    cor.append(1)
  else:
    cor.append(0)
result['Positivity'] = cor       #positive column having values 0 and 1 depicting positive and negative class used as ground truth  
cols = [ 'Score']
result.drop(cols, axis=1, inplace=True)
result.head()

,Reviews,Positivity
0,bought sever vital can dog food product found ...,1
1,product arriv label jumbo salt peanut peanut a...,0
2,confect around centuri light pillowi citru gel...,1
3,look secret ingredi robitussin believ found go...,0
4,great taffi great price wide assort yummi taff...,1


In [ ]:
result.groupby('Positivity').size()   #check the size of positive and negtive : they are in ration 1:3 so balnced

Positivity
0    11582
1    38418
dtype: int64

Test train:

In [ ]:
from sklearn.model_selection import train_test_split
X = result.Reviews         #put train data as alll review values of result 
y = result.Positivity       #all the labels are assign to y
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)    #test train split

In [ ]:
a=(len(X_train[y_train == 0]) / (len(X_train)*1.))*100
b=(len(X_train[y_train == 1]) / (len(X_train)*1.))*100
print("Overall train data ",len(X_train) ,"entries," ,a,"negativ response and ",b," positive response")

Overall train data  37500 entries, 23.106666666666666 negativ response and  76.89333333333333  positive response


In [ ]:
c=(len(X_test[y_test == 0]) / (len(X_test)*1.))*100
d=(len(X_test[y_test == 1]) / (len(X_test)*1.))*100
# print("overall test data : {0} entries , {1:.2f}% negative response and, {2:.2f}% positive response".format(len(X_test),c,d)) 
print("Overall train data ",len(X_test) ,"entries," ,c,"negativ response and ",d," positive response")                                                                      

Overall train data  12500 entries, 23.336000000000002 negativ response and  76.664  positive response


**MODEL:**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [ ]:
def accuracy_summary(pipeline, X_train, y_train, X_test, y_test):       #it will calculate accurcy precision recall and return the accurcay
    sentiment_fit = pipeline.fit(X_train, y_train)              #train model
    y_pred = sentiment_fit.predict(X_test)                      #predict the test data
    accuracy = accuracy_score(y_test, y_pred)                #take out accurcay
    cm=confusion_matrix(y_test, y_pred)            #make confusion matric
    tn, fp, fn, tp = cm.ravel()
    pre=tp/(tp+fp)   #precision
    recall=tp/(tp+fn)   #recall
    f1_score=(2*pre*recall)/(pre+recall) #f1score
    print("PRECISION,RECALL and F1-SCORE: ",pre,',',recall,',',f1_score)
    print("accuracy score: ",accuracy*100)
    return accuracy

In [ ]:

def avg_accuracy(feature_result_tgt):   #function take out average of all accurcies when we apply cross validation
  avg_acc=0
  for i in feature_result_tgt:
    avg_acc+=i[1]
  return avg_acc/len(feature_result_tgt)

Random forest:

In [ ]:
cv = CountVectorizer()     #countervectorization model for the text data
rf = RandomForestClassifier(class_weight="balanced")   #Random forst model
n_features = np.arange(10000,25001,5000)    # feature selection in range of features from 10000 to 25001 with cv of 5000 in each

#will return the n_feature accurcay and will takr input ngrams ranges from 1 to 3 as per require
def nfeature_accuracy_checker(vectorizer=cv, n_features=n_features, stop_words=None, ngram_range=(1, 3), classifier=rf):
    result = []
    print(classifier)
    print("\n")
    print("ngram: ", ngram_range)
    for n in n_features:         #will call accurcay_summary function that return accurcay for all the 5 different data set
        vectorizer.set_params(stop_words=stop_words, max_features=n, ngram_range=ngram_range)   #set vectorizer and put both 
        checker_pipeline = Pipeline([                                                           #vectorizer as well as classifier in one pipeline
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        print("Test result for {} features".format(n))
        nfeature_accuracy = accuracy_summary(checker_pipeline, X_train, y_train, X_test, y_test)
        result.append((n,nfeature_accuracy))
    return result    #return list of accuracies

In [ ]:
print("Result for trigram with stop words (Tfidf)\n")
feature_result_tgt = nfeature_accuracy_checker(tfidf,ngram_range=(1, 3))  #random forst for tfidf

Result for trigram with stop words (Tfidf)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


ngram:  (1, 3)
Test result for 10000 features
PRECISION,RECALL and F1-SCORE:  0.8589120264195945 , 0.9770426797453825 , 0.9141769185705917
accuracy score:  85.936
Test result for 15000 features
PRECISION,RECALL and F1-SCORE:  0.8580951511277509 , 0.9805906292392779 , 0.9152624914775495
accuracy score:  86.08
Test result for 20000 features
PRECISION,RECALL and F1-SCORE:  0.8560868771673663 , 0.97892

In [ ]:
print("average accurcay is:",avg_accuracy(feature_result_tgt))  #get avg accuracies

average accurcay is: 0.85902


In [ ]:
# .countervectorization:
feature_result_tgt = nfeature_accuracy_checker(cv,ngram_range=(1, 3))
print("average accurcay is:",avg_accuracy(feature_result_tgt))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


ngram:  (1, 3)
Test result for 10000 features
PRECISION,RECALL and F1-SCORE:  0.8615980808267208 , 0.9744338933528123 , 0.9145487488369816
accuracy score:  86.04
Test result for 15000 features
PRECISION,RECALL and F1-SCORE:  0.8562443026435734 , 0.9801732234164666 , 0.9140271493212669
accuracy score:  85.86399999999999
Test result for 20000 features
PRECISION,RECALL and F1-SCORE:  0.8571688832209874 , 0.9819471981634144 , 0.9153251301006761


Multinomial naive bayes classifer:

In [ ]:
# multinomial naive bayes:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

In [ ]:
feature_result_tgt = nfeature_accuracy_checker(cv,ngram_range=(1, 3),classifier=clf)  #calling to train multinomial and predict output for cv

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


ngram:  (1, 3)
Test result for 10000 features
PRECISION,RECALL and F1-SCORE:  0.911070780399274 , 0.8905353229677554 , 0.9006860158311345
accuracy score:  84.944
Test result for 15000 features
PRECISION,RECALL and F1-SCORE:  0.9159410942706654 , 0.8891787540436189 , 0.9023615376469343
accuracy score:  85.248
Test result for 20000 features
PRECISION,RECALL and F1-SCORE:  0.9153705888662448 , 0.8905353229677554 , 0.902782185549561
accuracy score:  85.296
Test result for 25000 features
PRECISION,RECALL and F1-SCORE:  0.9158378783017859 , 0.8936658666388396 , 0.9046160346466674
accuracy score:  85.55199999999999


In [ ]:
feature_result_tgt = nfeature_accuracy_checker(tfidf,ngram_range=(1, 3),classifier=clf)  #multinomial train model for tf-idf vectorizer

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


ngram:  (1, 3)
Test result for 10000 features
PRECISION,RECALL and F1-SCORE:  0.8352606297811778 , 0.9798601690493582 , 0.9018007202881152
accuracy score:  83.64
Test result for 15000 features
PRECISION,RECALL and F1-SCORE:  0.8332447750619908 , 0.9818428467077116 , 0.9014610778443114
accuracy score:  83.544
Test result for 20000 features
PRECISION,RECALL and F1-SCORE:  0.8292296869504046 , 0.9840342272774705 , 0.9000238606537819
accuracy score:  83.24000000000001
Test result for 25000 features
PRECISION,RECALL and F1-SCORE:  0.8251699494509326 , 0.9879995825941772 , 0.8992734007693404
accuracy score:  83.032


KNN model with neighor 7

In [ ]:
#  KNN 
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=7)

In [ ]:
feature_result_tgt = nfeature_accuracy_checker(cv,ngram_range=(1, 3),classifier=neigh)  #KNN train model for count vectorizer

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')


ngram:  (1, 3)
Test result for 10000 features
PRECISION,RECALL and F1-SCORE:  0.7947060365161177 , 0.9492851925284358 , 0.8651450309082264
accuracy score:  77.312
Test result for 15000 features
PRECISION,RECALL and F1-SCORE:  0.7969884853852968 , 0.9389543984138579 , 0.8621664351075553
accuracy score:  76.984
Test result for 20000 features
PRECISION,RECALL and F1-SCORE:  0.7988283010365029 , 0.9248669518939789 , 0.8572395782957734
accuracy score:  76.384
Test result for 25000 features
PRECISION,RECALL and F1-SCORE:  0.7973359556588593 , 0.9307106334133362 , 0.8588762097356637
accuracy score:  76.55199999999999


In [ ]:
print("average accurcay is:",avg_accuracy(feature_result_tgt))       #take out avg accurcis with the above

average accurcay is: 0.76808


In [ ]:
feature_result_tgt = nfeature_accuracy_checker(tfidf,ngram_range=(1, 3),classifier=neigh) #KNN train model for tf-idf vectorizer
print("average accurcay is:",avg_accuracy(feature_result_tgt))    #average accurcies for the same

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')


ngram:  (1, 3)
Test result for 10000 features
PRECISION,RECALL and F1-SCORE:  0.8162544169611308 , 0.9401022644265887 , 0.8738118331716781
accuracy score:  79.184
Test result for 15000 features
PRECISION,RECALL and F1-SCORE:  0.8163598440192256 , 0.9393718042366691 , 0.8735565259582727
accuracy score:  79.152
Test result for 20000 features
PRECISION,RECALL and F1-SCORE:  0.8158658204895739 , 0.9390587498695607 , 0.8731383107747539
accuracy score:  79.08
Test result for 25000 features
PRECISION,RECALL and F1-SCORE:  0.8186185641769398 , 0.9423979964520505 , 0.8761581372786805
accuracy score:  79.57600000000001
average accurcay is: 0.79248


Random forest bagging classifier:

In [ ]:
from sklearn.ensemble import BaggingClassifier        #bagging classifier with random forest
bagging = BaggingClassifier(rf,
                            max_samples=0.5, max_features=0.5)

In [ ]:
feature_result_tgt = nfeature_accuracy_checker(cv,ngram_range=(1, 3),classifier=bagging)
print("average accurcay is:",avg_accuracy(feature_result_tgt))

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight='balanced',
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,

39758    treat cat eat love wish amazon favor sinc mani...
18457    also purchas pot soil repot plant home gnat ev...
33239    sumatra full syrupi bodi virtual acid coffe in...
35006    realli want like product sadli hate br br text...
30982    thought would good substitut regular salt vine...
                               ...                        
21243    got teeni greeni lb shih tzu lhasa apso packag...
45891    love tast ginger lemon realli enjoy drink caff...
42613    well ye product fine good price receiv amount ...
43567    almond yummi order onlin store alway walmart t...
2732     popcorn grear color white tast averag due ship...
Name: Reviews, Length: 37500, dtype: object

In [ ]:
# data = {'Test':["Hello how are you?"]}
# df = pd.DataFrame(data) 
# df

,Test
0,Hello how are you?


**Natural Language interface for user to write an statment ans check whether the commented review is positive or negative:**

In [ ]:
def nlp(pipeline, X_train, y_train, X_test,y_test,new_data):  #this for responding the new dataset which ask by the user randomly
    sentiment_fit = pipeline.fit(X_train, y_train)     #train the model
    y_pred = sentiment_fit.predict(X_test)     #predict the validation
    accuracy = accuracy_score(y_test, y_pred)    #take out acvcurcay of validation
    cm=confusion_matrix(y_test, y_pred)  #form confusion matrix of validation
    tn, fp, fn, tp = cm.ravel()
    pre=tp/(tp+fp)     #precision
    recall=tp/(tp+fn)    #recall
    f1_score=(2*pre*recall)/(pre+recall)  #f1-score
    print("PRECISION,RECALL and F1-SCORE: ",pre,',',recall,',',f1_score)
    print("accuracy score: ",accuracy*100)

    y_pred_new_data = sentiment_fit.predict(new_data)   #prediction for new data set
    return y_pred_new_data    #return the 0 or 1 can further take as negatibve or positive classification



def call_me(vectorizer=cv, n_features=n_features, stop_words=None, ngram_range=(1, 3), classifier=rf):  #it will call nlp() to take out values of evalutation metric
    result = []
    print(classifier)
    print("\n")
    print("ngram: ", ngram_range)
    n=n_features[3]+n_features[2]  #here i take totoal 40000 features 
    vectorizer.set_params(stop_words=stop_words, max_features=n, ngram_range=ngram_range)   #take vectorizer and classifier into pipeline and train them
    checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
    print("Test result for {} features".format(n))
    inp=input("Enter the review to check whether it is positive or negative:\n")  #from here take input from the user
    data = {'Test':[inp]}    #form dataset of new data 
    df = pd.DataFrame(data)    
    nfeature_accuracy = nlp(checker_pipeline, X_train, y_train, X_test,y_test,data)   #call for nlp() to predict
    return nfeature_accuracy

Voting classfier: with having random forest, random forest bagging, and knn as their classsifiers to be voted.

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
eclf1 = VotingClassifier(estimators=[
        ('lr', neigh), ('rf', rf), ('RF_BAGGING', bagging)], voting='hard')

In [ ]:
feature_result_tgt = call_me(cv,ngram_range=(1, 3),classifier=eclf1)   #call the interface calling funtion to return the response for count vectorizer
if feature_result_tgt>=1:        #to check the rsponse if it is 0 then negative response else positive response.
  print("Positive Response")
else:
  print("Negative Response")

VotingClassifier(estimators=[('lr',
                              KNeighborsClassifier(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='minkowski',
                                                   metric_params=None,
                                                   n_jobs=None, n_neighbors=7,
                                                   p=2, weights='uniform')),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight='balanced',
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes

In [ ]:
if feature_result_tgt>=1:  #to check the rsponse if it is 0 then negative response else positive response.
  print("Positive Response")
else:
  print("Negative Response")

Positive Response


In [ ]:
feature_result_tgt = call_me(tfidf,ngram_range=(1, 3),classifier=eclf1)  #call the interface calling funtion to return the response for tf-idf vectorizer
if feature_result_tgt>=1:   #to check the rsponse if it is 0 then negative response else positive response.
  print("Positive Response")
else:
  print("Negative Response")

VotingClassifier(estimators=[('lr',
                              KNeighborsClassifier(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='minkowski',
                                                   metric_params=None,
                                                   n_jobs=None, n_neighbors=7,
                                                   p=2, weights='uniform')),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight='balanced',
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes

## Feature Engineering

In [ ]:
df_review=pd.read_csv('/content/drive/MyDrive/cgas_dataset/Reviews.csv') #read the train file

In [ ]:
df_rating=pd.DataFrame(df_review['Text'])       #take Textcolumn in df_rating and  
df_rating=df_rating.fillna('')                  #fill " " in all nan values

In [ ]:
df_rating.columns      #df_rating have only Text  column

Index(['Text'], dtype='object')

In [ ]:
r=[]
for i in df_review['Score']:    #label encoding of Score column if value greater then 3 then let it 1 else 0
  if i>3:
    r.append(1)
  else:
    r.append(0)

  

In [ ]:
df_rating['Sentiment']=r       #create new column Sentiment and assign that new label encoding 0 and 1 to it .

In [ ]:
df_rating.groupby('Sentiment').size() #check for Sentiment column  size

Sentiment
0    124677
1    443777
dtype: int64

In [ ]:
import nltk     #import all the ntlk liberaries
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Remove extra spaces from text data.
Used NLTK Word tokenizer to generate tokens from text data.
Remove stopwords( used nltk stop word list) from token lists.
Use nltk word lamatizer to lamatize the tokens.


In [ ]:
def preprocessing(sen):
  # result=sen.lower()
  result = re.sub(r"\d+", "", result)
  stop_words = set(stopwords.words('english'))
  tokens = nltk.word_tokenize(result)
  result = [i for i in tokens if not i in stop_words]
  lemmatizer=nltk.WordNetLemmatizer()
  doc=[]
  for word in result:
    word1=lemmatizer.lemmatize(word)
    doc.append(word1)
  return doc
  

We have calculated the number of occurrences of each partof-speech tag in each review.


In [ ]:
def pos(sen):
    temp={}
    tag=set()
#     tokens=nltk.word_tokenize(sen)
    for w1 in sen:
        w1=re.sub('[!@#$%^&*()|\/,.:;<>-?]','',w1)
    pos_=nltk.pos_tag(sen)
    for pair in pos_:
        if pair[1] not in temp:
            temp[pair[1]]=1
    return temp
    

The number of contiguous sequences of exclamation marks, question marks, and both exclamation and question marks present in review.
 Whether the last token contains an exclamation or question mark.


In [ ]:
def punctuation(sen):
    c=0

    for word in sen:
        if re.search('[!?]+',word):
            c+=1
    pl=0
    if len(sen)>0 and re.search('[!?]+',sen[-1]):
        pl=1
    return c,pl

Presence or absence of positive and negative emoticons at any position in the review,  whether the last token is a positive or negative emoticon.
(The polarity of an emoticon was determined with a regular expression adopted from Christopher Potts’ tokenizing script)


In [ ]:
def emoticons(sen):
  emoticon_string = r"""
    (?:
      [<>]?
      [:;=8]                     
      [\-o\*\']?                 
      [\)\]\(\[dDpP/\:\}\{@\|\\]     
      |
      [\)\]\(\[dDpP/\:\}\{@\|\\] 
      [\-o\*\']?                 
      [:;=8]                     
      [<>]?
    )"""
  emojies=re.findall(emoticon_string,sen)
  temp={}
  for em in emojies:
    if em not in temp:
      temp[em]=1
  l_em=0
  if re.search(emoticon_string,sen[-1]):
    l_em=1
  return temp,l_em

The number of words with one character repeated more than two times, for example, ‘Gooood’.


In [ ]:
def elongation(sen):
    elong_string="([a-zA-Z])\\1{2,}"
    c=0
#     sen=sen.split(' ')
    for w in sen:
        if re.search(elong_string,w):
            c+=1
    return c
    
        

The number of negated contexts present in review.

In [ ]:
def negation(sen):
    negation_se='(?:^(?:never|no|nothing|nowhere|noone|none|not|havent|hasnt|hadnt|cant|couldnt|shouldnt|wont|wouldnt|dont|doesnt|didnt|isnt|arent|aint)$)|n\'t'
#     sen=sen.split()
    c=0
    for i in range(len(sen)-1):
        if re.search(negation_se,sen[i]):
            c+=1
    return c
        

The number of words with all characters in upper case;


In [ ]:
def allcap(sen):
#     sen=re.sub('_-',' ',sen)
#     sen=sen.split()
    cap='^[#@$\n]*[A-Z]+[.?!-_,.]*[A-Z]*$'
    c=0
    for word in sen:
        if re.search(cap,word):
            c+=1
    return c
    

In [ ]:
import pickle
def store(data,file1):
    outfile = open(file1,'wb')
    pickle.dump(data,outfile)
    outfile.close()
def load(file1):
    infile = open(file1,'rb')
    data_loaded = pickle.load(infile)
    return data_loaded

Traing


presence or absence of contiguous sequences of 1, 2 tokens; noncontiguous ngrams (ngrams with one token replaced by *). For noncontiguous we considered only bigram.(we used nltk ngram and skip gram library to generate contiguous and noncontiguous n gram respectively)


In [ ]:
#other Features
def unigrams(id1,sen):
  global all_unigram
  global unigram
  sen1=[]
  for i in sen:
    i=re.sub('[!@#$%^&*()|\/,.:;<>-?\'\"]','',i)
    sen1.append(i)
  sen=sen1
  unigram_=list(nltk.ngrams(sen,1))
  temp={}
  for col in unigram_:
    temp[col]=1
    try:
      all_unigram[col]+=1
    except:
      all_unigram[col]=1
  unigram[id1]=temp

In [ ]:
def skipgrams(ind,sen):
  global all_ngram
  global ngram
  sen1=[]
  for i in sen:
        i=re.sub('[!@#$%^&*()|\/,.:;<>-?\'\"]','',i)
        sen1.append(i)
  sen=sen1
  skip_bigram=list(nltk.skipgrams(sen,2,4))
    # skip_trigram=list(nltk.skipgrams(sen,3,4))
  temp={}
  for col in skip_bigram:
      temp[col]=1
      try:
        all_ngram[col]+=1
      except:
        all_ngram[col]=1
  ngram[ind]=temp
    # temp={}
    # for col in skip_trigram:
    #     temp[col]=1
    #     if col not in all_ngrams_:
    #          all_ngrams_[col]=0
    #     all_ngrams_[col]+=1
    # ngrams_[ind]=temp

In [ ]:
all_ngram={}
all_unigram={}
unigram={}
ngram={}
n_other={}
features={}
df=df_rating
gt={}
df=df_rating[:150000]
for ind in df.index:
  sen1=df['Text'][ind]
  sen= preprocessing(sen1)
  if(len(sen)>15 or len(sen)<1):
    continue
  gt[ind]=df['Sentiment'][ind]
  features[ind]={}
  features[ind].update(pos(sen))
  features[ind]['punc_count'],features[ind]['last_punc']=punctuation(sen)
  temp,l_em=emoticons(sen1)
  features[ind].update(temp)
  features[ind]['last_emo']=l_em
  features[ind]['elongation']=elongation(sen)
  features[ind]['negations']=negation(sen)
  features[ind]['cap']=allcap(sen)
  unigrams(ind,sen)
  skipgrams(ind,sen)

  


In [ ]:
selected_unigram={key:val for key, val in all_unigram.items() if val > 50}
selected_ngram={key:val for key, val in all_ngram.items() if val > 50}
new_unigram={}
new_ngram={}
for key in unigram.keys():
    temp={}
    for z in unigram[key].keys():
        if z in selected_unigram.keys():
            temp[z]=1
    new_unigram[key]=temp
for key in ngram.keys():
    temp={}
    for z in ngram[key].keys():
        if z in selected_ngram.keys():
            temp[z]=1
    new_ngram[key]=temp

In [ ]:
# features.update(new_unigram)
# features.update(new_ngram)


In [ ]:
store(new_ngram,'new_ngram.csv')
store(new_unigram,'new_unigram.csv')

In [ ]:
other_feature=pd.DataFrame(features)
other_feature=other_feature.transpose()
other_feature


,JJ,VBP,RB,VBG,NN,.,VB,IN,punc_count,last_punc,last_emo,elongation,negations,cap,VBD,VBN,:,NNS,CD,",",MD,POS,VBZ,JJS,JJR,RBR,CC,),FW,$,DT,NNP,(,SYM,EX,``,'',RP,UH,RBS,PRP,WDT,WRB,TO,PRP$,#,WP
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,1.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,3.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,1.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,1.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,1.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149928,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149930,1.0,1.0,1.0,NaN,1.0,1.0,1.0,NaN,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149933,1.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149984,1.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
other_feature=other_feature.fillna(0)
other_feature

,JJ,VBP,RB,VBG,NN,.,VB,IN,punc_count,last_punc,last_emo,elongation,negations,cap,VBD,VBN,:,NNS,CD,",",MD,POS,VBZ,JJS,JJR,RBR,CC,),FW,$,DT,NNP,(,SYM,EX,``,'',RP,UH,RBS,PRP,WDT,WRB,TO,PRP$,#,WP
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149928,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149930,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149933,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149984,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ngram_df=pd.DataFrame(new_ngram)
ngram_df=ngram_df.transpose()
unigram_df=pd.DataFrame(new_unigram)
unigram_df=unigram_df.transpose()
ngram_feature=pd.concat([ngram_df,unigram_df],axis=1)

In [ ]:
ngram_feature=ngram_feature.fillna(0)
ngram_feature

,"(cat, )","(cat, love)","(eat, )","(, love)","(, )","(, around)","(love, )","(good, flavor)","(good, )","(flavor, )","(, came)","(, fresh)","(came, )","(packed, )","(, delicious)","(fresh, )","(delicious, )","(-, )","(yummy, )","(, son)","(around, )","(son, )","(satisfied, )","(purchase, )","(, others)","(, enjoyed)","(, definitely)","(others, )","(enjoyed, )","(, ordering)","(definitely, )","(ordering, )","(delivered, )","(fast, )","(purchased, )","(reasonable, )","(price, )","(, home)","(, get)","(, store)",...,"(care,)","(aroma,)","(breakfast,)","(protein,)","(cake,)","(online,)","(butter,)","(decaf,)","(beef,)","(seems,)","(baby,)","(seasoning,)","(jerky,)","(seed,)","(peach,)","(lb,)","(needed,)","(making,)","(energy,)","(start,)","(peanut,)","(french,)","(least,)","(enjoys,)","(training,)","(honey,)","(ginger,)","(drinking,)","(cheese,)","(greenies,)","(roast,)","(usually,)","(wake,)","(puppy,)","(place,)","(bone,)","(starbucks,)","(chai,)","(apple,)","(seller,)"
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149930,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149984,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0

In [ ]:
train_features=pd.concat([other_feature,ngram_feature],axis=1)
train_features

,JJ,VBP,RB,VBG,NN,.,VB,IN,punc_count,last_punc,last_emo,elongation,negations,cap,VBD,VBN,:,NNS,CD,",",MD,POS,VBZ,JJS,JJR,RBR,CC,),FW,$,DT,NNP,(,SYM,EX,``,'',RP,UH,RBS,...,"(care,)","(aroma,)","(breakfast,)","(protein,)","(cake,)","(online,)","(butter,)","(decaf,)","(beef,)","(seems,)","(baby,)","(seasoning,)","(jerky,)","(seed,)","(peach,)","(lb,)","(needed,)","(making,)","(energy,)","(start,)","(peanut,)","(french,)","(least,)","(enjoys,)","(training,)","(honey,)","(ginger,)","(drinking,)","(cheese,)","(greenies,)","(roast,)","(usually,)","(wake,)","(puppy,)","(place,)","(bone,)","(starbucks,)","(chai,)","(apple,)","(seller,)"
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149928,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149930,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149933,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149984,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Test

In [ ]:
#other Features
def unigrams__(id1,sen):
  # global all_unigram
  global unigram_
  sen1=[]
  for i in sen:
    i=re.sub('[!@#$%^&*()|\/,.:;<>-?\'\"]','',i)
    sen1.append(i)
  sen=sen1
  unigram__=list(nltk.ngrams(sen,1))
  temp={}
  for col in unigram__:
    if col in selected_unigram:
      temp[col]=1
  unigram_[id1]=temp

In [ ]:
def skipgrams_(ind,sen):
  # global all_ngram
  global ngram_
  sen1=[]
  for i in sen:
        i=re.sub('[!@#$%^&*()|\/,.:;<>-?\'\"]','',i)
        sen1.append(i)
  sen=sen1
  skip_bigram=list(nltk.skipgrams(sen,2,4))
    # skip_trigram=list(nltk.skipgrams(sen,3,4))
  temp={}
  for col in skip_bigram:
    if col in selected_ngram:
      temp[col]=1
    ngram_[ind]=temp
    # temp={}
    # for col in skip_trigram:
    #     temp[col]=1
    #     if col not in all_ngrams_:
    #          all_ngrams_[col]=0
    #     all_ngrams_[col]+=1
    # ngrams_[ind]=temp

In [ ]:
unigram_={}
ngram_={}
features_test={}
gt_test={}
df=df_rating[150000:300000]
for ind in df.index:
  sen1=df['Text'][ind]
  sen= preprocessing(sen1)
  if(len(sen)>15 or len(sen)<1):
    continue
  gt_test[ind]=df['Sentiment'][ind]
  features_test[ind]={}
  features_test[ind].update(pos(sen))
  features_test[ind]['punc_count'],features_test[ind]['last_punc']=punctuation(sen)
  temp,l_em=emoticons(sen1)
  features_test[ind].update(temp)
  features_test[ind]['last_emo']=l_em
  features_test[ind]['elongation']=elongation(sen)
  features_test[ind]['negations']=negation(sen)
  features_test[ind]['cap']=allcap(sen)
  unigrams__(ind,sen)
  skipgrams_(ind,sen)

  


In [ ]:
feature_test=pd.DataFrame(features_test)
feature_test=feature_test.transpose()
feature_test=feature_test.fillna(0)
unigram_df=pd.DataFrame(unigram_)
unigram_df=unigram_df.transpose()
unigram_df=unigram_df.fillna(0)
ngram_df=pd.DataFrame(ngram_)
ngram_df=ngram_df.transpose()
ngram_df=ngram_df.fillna(0)

In [ ]:
unigram_df

,arrived,quickly,perfect,condition,,definitely,look,m,love,bar,wish,could,get,local,store,minute,half,add,milk,mix,delicious,fantastic,product,kid,make,tasty,snack,nt,take,many,day,s,calorie,taste,great,expensive,though,want,food,good,...,enjoys,decaf,wake,waste,k-cup,mom,recipe,peach,another,recommended,seller,weight,potato,review,subscribe,usually,seed,extremely,chip,stale,apple,needed,gum,soft,training,dark,salad,cake,lemon,bold,dried,wait,type,soon,took,french,vanilla,vet,office,greenies
150039,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150059,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150061,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150105,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150147,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299985,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299988,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299990,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299996,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test_features=pd.concat([feature_test,unigram_df,ngram_df],axis=1)
test_features

,JJ,RB,NN,.,VBP,VBG,punc_count,last_punc,last_emo,elongation,negations,cap,VB,MD,VBN,",",NNS,$,POS,IN,VBD,VBZ,``,'',:,(,),CD,RBR,JJS,RBS,JJR,CC,NNP,DT,FW,UH,PRP,RP,EX,...,"(addictive, )","(, dark)","(make, good)","(good, cup)","(door, )","(color, )","(, star)","(coffee, bold)","(bold, )","(, soft)","(dried, )","(salad, )","(wait, )","(soon, )","(, half)","(easier, )","(peanut, butter)","(, rich)","(french, )","(vanilla, )","(nice, flavor)","(type, )","(, case)","(, gave)","(aftertaste, )","(birthday, )","(pie, )","(mine, )","(, soon)","(stale, )","(wow, )","(, vanilla)","(flavor, flavor)","(flavored, )","(, bold)","(, cheese)","(, peanut)","(office, )","(hour, )","(greenies, )"
150039,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150059,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150061,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150105,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150147,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299985,1.0,1.0,1.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299988,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299990,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299996,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
store(training_feature)

Lexicon

We have taken NRC Emotion Lexicon, MPQA, Bing Liu Lexicon script. These scripts contain words and it’s polarity in terms of different dimensions.
 So if any word from a review data present in the above mentioned script,  we take the  polarity of that word(for all dimensions), and in that way we have calculated how much overall polarity the review has. Suppose, “Good” and “less” these two words present in review data, and we get polarity score 1 for “Good” and “-1” for “less”, or get “GOOd” as positive polarity and “less” as negative polarity word.


In [ ]:
file=open('/content/drive/MyDrive/cgas_dataset/lexicon.txt')
#lexicon is NRC word emotion
file=file.read()
lex={}
polarity1=set()
file=file.split('\n')
for line in file:
    t=line.split('\t')
    polarity1=polarity1|{t[1]}
    if t[0] in lex:
        lex[t[0]][t[1]]=t[2]
    else:
        lex[t[0]]={}
        lex[t[0]][t[1]]=t[2]
polarity1=list(polarity1) 
def lexicon(sen):
    global lex
    global polarity1
    temp={k:0 for k in polarity1}
    total=0
    for p in polarity1:
        t=0
        for w in sen:
            if w in lex:
#                 print(p,lex[word][p])
                t+=int(lex[w][p])
        temp[p]+=t
                
    return temp      

In [ ]:
df_mpqa=pd.read_csv('/content/drive/MyDrive/cgas_dataset/MPQA.csv')
dict_mpqa={}
for index, row in df_mpqa.iterrows():
    dict_mpqa[row['Word']]=row['Polarity']
def mpqa(sen):
    temp={'positive_mpqa':0,'negative_mpqa':0}
    for w in sen:
        if w in dict_mpqa:
            if dict_mpqa[w]=='positive':
                temp['positive_mpqa']+=1
            else:
                temp['negative_mpqa']+=1
    return temp
    

In [ ]:
df_mpqa

,Word,Polarity
0,abandoned,negative
1,abandonment,negative
2,abandon,negative
3,abase,negative
4,abasement,negative
...,...,...
8216,zealot,negative
8217,zealous,negative
8218,zealously,negative
8219,zenith,positive


In [ ]:
lex

{'aback': {'anger': '0',
  'anticipation': '0',
  'disgust': '0',
  'fear': '0',
  'joy': '0',
  'negative': '0',
  'positive': '0',
  'sadness': '0',
  'surprise': '0',
  'trust': '0'},
 'abacus': {'anger': '0',
  'anticipation': '0',
  'disgust': '0',
  'fear': '0',
  'joy': '0',
  'negative': '0',
  'positive': '0',
  'sadness': '0',
  'surprise': '0',
  'trust': '1'},
 'abandon': {'anger': '0',
  'anticipation': '0',
  'disgust': '0',
  'fear': '1',
  'joy': '0',
  'negative': '1',
  'positive': '0',
  'sadness': '1',
  'surprise': '0',
  'trust': '0'},
 'abandoned': {'anger': '1',
  'anticipation': '0',
  'disgust': '0',
  'fear': '1',
  'joy': '0',
  'negative': '1',
  'positive': '0',
  'sadness': '1',
  'surprise': '0',
  'trust': '0'},
 'abandonment': {'anger': '1',
  'anticipation': '0',
  'disgust': '0',
  'fear': '1',
  'joy': '0',
  'negative': '1',
  'positive': '0',
  'sadness': '1',
  'surprise': '1',
  'trust': '0'},
 'abate': {'anger': '0',
  'anticipation': '0',
  'd

In [ ]:
import pandas as pd
df_binglu_=pd.read_csv('/content/drive/MyDrive/cgas_dataset/BingLiu.csv')
word=list(df_binglu_['Word'])
polarity=list(df_binglu_['Polarity'])
def binglu(sen):
    global word
    global polarity
    temp={'positive':0,'negative':0}
    for w in sen:
        if w in word:
            ind=word.index(w)
            temp[polarity[ind]]+=1
    return temp

# df_binglu


In [ ]:
df_binglu_

,Word,Polarity
0,a+,positive
1,abound,positive
2,abounds,positive
3,abundance,positive
4,abundant,positive
...,...,...
6784,zaps,negative
6785,zealot,negative
6786,zealous,negative
6787,zealously,negative


In [ ]:
def agg_polarityScore(sent):
  # sent = remove_hash(sent)
    sentScore = {}
    pos = 0
    neg = 0
    for token in sent:
        try:
            score = word_sentiment_score[token]
            if(score<0):
                neg +=score
            else:
                pos +=score
        except:
            pass

    pos = pos/len(sent)
    neg = neg/len(sent)
    sentScore['positive_agg'] = pos
    sentScore['negative_agg'] = neg
    return sentScore

In [ ]:
dict_binglu_train={}
dict_lex_train={}
dict_nrc_punc_train={}
# dict_hash_emotion_score_train={}
dict_polarityagg_train={}
dict_mpqa_train={}
df=df_rating[:150000]
for ind in df.index:
    sen=preprocessing(df['Text'][ind])
    if(len(sen)>15 or len(sen)<1):
        continue
    dict_binglu_train[ind]=binglu(sen)
    dict_lex_train[ind]=lexicon(sen)
    # dict_hash_emotion_score_train[df2['1'][ind]]=polarity_hash(df2['5'][ind])
    dict_polarityagg_train[ind]=agg_polarityScore(sen)
    dict_mpqa_train[ind]=mpqa(sen)
    
    

In [ ]:
df_binglu_train=pd.DataFrame(dict_binglu_train)
df_binglu_train=df_binglu_train.transpose()
df_lex_train=pd.DataFrame(dict_lex_train)
df_lex_train=df_lex_train.transpose()
df_polarityagg_train=pd.DataFrame(dict_polarityagg_train)
df_polarityagg_train=df_polarityagg_train.transpose()
df_mpqa_train=pd.DataFrame(dict_mpqa_train)
df_mpqa_train=df_mpqa_train.transpose()

In [ ]:
df_lexicon_train=pd.concat([df_binglu_train,df_lex_train,df_polarityagg_train,df_mpqa_train],axis=1)
df_lexicon_train

,positive,negative,surprise,joy,positive,trust,negative,fear,sadness,anticipation,anger,disgust,positive_agg,negative_agg,positive_mpqa,negative_mpqa
8,2,0,0,1,2,0,0,0,0,0,0,0,0.0,0.0,2,1
13,5,0,1,3,3,1,0,0,0,1,0,0,0.0,0.0,5,0
14,1,1,0,0,0,0,2,0,1,0,2,0,0.0,0.0,1,1
17,2,0,0,1,1,0,0,0,0,0,0,0,0.0,0.0,1,0
19,3,1,0,1,2,1,2,0,1,2,0,0,0.0,0.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149928,2,0,0,1,2,1,0,0,0,1,0,0,0.0,0.0,2,0
149930,1,0,0,1,1,0,0,0,0,1,0,0,0.0,0.0,1,3
149933,2,0,1,0,1,0,0,0,0,0,0,0,0.0,0.0,2,0
149984,1,0,1,2,3,2,0,0,0,1,0,0,0.0,0.0,3,0


In [ ]:
df_train=pd.concat([df_lexicon_train,train_features],axis=1)
df_train


,positive,negative,surprise,joy,positive,trust,negative,fear,sadness,anticipation,anger,disgust,positive_agg,negative_agg,positive_mpqa,negative_mpqa,JJ,VBP,RB,VBG,NN,.,VB,IN,punc_count,last_punc,last_emo,elongation,negations,cap,VBD,VBN,:,NNS,CD,",",MD,POS,VBZ,JJS,...,"(care,)","(aroma,)","(breakfast,)","(protein,)","(cake,)","(online,)","(butter,)","(decaf,)","(beef,)","(seems,)","(baby,)","(seasoning,)","(jerky,)","(seed,)","(peach,)","(lb,)","(needed,)","(making,)","(energy,)","(start,)","(peanut,)","(french,)","(least,)","(enjoys,)","(training,)","(honey,)","(ginger,)","(drinking,)","(cheese,)","(greenies,)","(roast,)","(usually,)","(wake,)","(puppy,)","(place,)","(bone,)","(starbucks,)","(chai,)","(apple,)","(seller,)"
8,2,0,0,1,2,0,0,0,0,0,0,0,0.0,0.0,2,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,5,0,1,3,3,1,0,0,0,1,0,0,0.0,0.0,5,0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,1,1,0,0,0,0,2,0,1,0,2,0,0.0,0.0,1,1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,2,0,0,1,1,0,0,0,0,0,0,0,0.0,0.0,1,0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,3,1,0,1,2,1,2,0,1,2,0,0,0.0,0.0,2,2,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149928,2,0,0,1,2,1,0,0,0,1,0,0,0.0,0.0,2,0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149930,1,0,0,1,1,0,0,0,0,1,0,0,0.0,0.0,1,3,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149933,2,0,1,0,1,0,0,0,0,0,0,0,0.0,0.0,2,0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149984,1,0,1,2,3,2,0,0,0,1,0,0,0.0,0.0,3,0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
store(df_train,'all_train_features.csv')

Test

In [ ]:
dict_binglu_test={}
dict_lex_test={}
dict_nrc_punc_test={}
dict_polarityagg_test={}
dict_mpqa_test={}
df=df_rating[150000:300000]
for ind in df.index:
    sen=preprocessing(df['Text'][ind])
    if(len(sen)>15 or len(sen)<1):
      continue
    dict_binglu_test[ind]=binglu(sen)
    dict_lex_test[ind]=lexicon(sen)
    if(len(sen)!=0):
        dict_polarityagg_test[ind]=agg_polarityScore(sen)
    else:
        dict_polarityagg_test[ind]={'positive_agg':0,'negative_agg':0}
    dict_mpqa_test[ind]=mpqa(sen)
    

In [ ]:
df_binglu_test=pd.DataFrame(dict_binglu_test)
df_binglu_test=df_binglu_test.transpose()
df_lex_test=pd.DataFrame(dict_lex_test)
df_lex_test=df_lex_test.transpose()
df_polarityagg_test=pd.DataFrame(dict_polarityagg_test)
df_polarityagg_test=df_polarityagg_test.transpose()
df_mpqa_test=pd.DataFrame(dict_mpqa_test)
df_mpqa_test=df_mpqa_test.transpose()

In [ ]:
df_lexicon_test=pd.concat([df_binglu_test,df_lex_test,df_polarityagg_test,df_mpqa_test],axis=1)

In [ ]:
df_test=pd.concat([df_lexicon_test,test_features],axis=1)
df_test

,positive,negative,surprise,joy,positive,trust,negative,fear,sadness,anticipation,anger,disgust,positive_agg,negative_agg,positive_mpqa,negative_mpqa,JJ,RB,NN,.,VBP,VBG,punc_count,last_punc,last_emo,elongation,negations,cap,VB,MD,VBN,",",NNS,$,POS,IN,VBD,VBZ,``,'',...,"(addictive, )","(, dark)","(make, good)","(good, cup)","(door, )","(color, )","(, star)","(coffee, bold)","(bold, )","(, soft)","(dried, )","(salad, )","(wait, )","(soon, )","(, half)","(easier, )","(peanut, butter)","(, rich)","(french, )","(vanilla, )","(nice, flavor)","(type, )","(, case)","(, gave)","(aftertaste, )","(birthday, )","(pie, )","(mine, )","(, soon)","(stale, )","(wow, )","(, vanilla)","(flavor, flavor)","(flavored, )","(, bold)","(, cheese)","(, peanut)","(office, )","(hour, )","(greenies, )"
150039,1,0,0,1,1,1,0,0,0,2,0,0,0.0,0.0,3,1,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150059,1,0,0,1,2,0,0,0,0,1,0,0,0.0,0.0,2,2,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150061,2,0,1,1,1,0,0,0,0,0,0,0,0.0,0.0,2,0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150105,2,0,0,0,2,0,0,0,0,1,0,0,0.0,0.0,3,0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150147,0,0,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0,0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299985,3,1,0,1,1,0,1,0,1,0,0,1,0.0,0.0,3,1,1.0,1.0,1.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299988,1,1,1,3,3,3,0,0,0,1,0,0,0.0,0.0,1,0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299990,2,0,1,1,1,1,1,1,1,1,1,1,0.0,0.0,2,0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299996,3,0,0,1,2,1,1,0,0,1,0,0,0.0,0.0,2,2,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Fit in model

In [ ]:
features=list(set(df_test.columns)&set(df_train.columns))
X_train=df_train[features]
X_train=X_train.fillna(0)
y_train=list(gt.values())
X_test=df_test[features]
X_test=X_test.fillna(0)
y_test=list(gt_test.values())

DECISION TREE

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
clf3 = DecisionTreeClassifier(random_state=0)
clf3.fit(X_train,y_train)
y_pred = clf3.predict(X_test)
cf_dt=confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cf_dt.ravel()
pre=tp/(tp+fp)
recall=tp/(tp+fn)
f1_score=(2*pre*recall)/(pre+recall)
print("PRECISION,RECALL and F1-SCORE: ",pre,',',recall,',',f1_score)
print("Accuracy of Decision Tree classifier = ",accuracy_score(y_test, y_pred))



PRECISION,RECALL and F1-SCORE:  0.9165961049957663 , 0.911271992591969 , 0.913926294904808
Accuracy of Decision Tree classifier =  0.8574325269193119


SVM Classifier

In [ ]:
from sklearn import svm
clf2 = svm.SVC()
clf2.fit(X_train,y_train)
y_pred=clf2.predict(X_test)
cf_svm=confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cf_svm.ravel()
pre=tp/(tp+fp)
recall=tp/(tp+fn)
f1_score=(2*pre*recall)/(pre+recall)
print("PRECISION,RECALL and F1-SCORE: ",pre,',',recall,',',f1_score)
print("Accuracy of SVM classifier = ",accuracy_score(y_test, y_pred))


PRECISION,RECALL and F1-SCORE:  0.901704105344694 , 0.9799646434885091 , 0.9392069062890799
Accuracy of SVM classifier =  0.8946301216613061


MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=100).fit(X_train, y_train)
# clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
cf_mlp=confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cf_mlp.ravel()
pre=tp/(tp+fp)
recall=tp/(tp+fn)
f1_score=(2*pre*recall)/(pre+recall)
print("PRECISION,RECALL and F1-SCORE: ",pre,',',recall,',',f1_score)
print("Accuracy of MLP classifier = ",accuracy_score(y_test, y_pred))


PRECISION,RECALL and F1-SCORE:  0.9294194710554328 , 0.9555518141257682 , 0.9423044994188943
Accuracy of MLP classifier =  0.9028107956929101


Naive Base 